## Bayesian inference analysis notebook

This is an example notebook about how to use our tools to analysis a observation constraint on neutron star equation of state. 

Here in this notebook, we are using a polytrope EoS model (in the future we will implement more equation of state)

In [ ]:
import InferenceWorkflow.prior as prior
import numpy as np
import ultranest
import ultranest.stepsampler

import EOSgenerators.Polytrope_EOS as Polytrope
from TOVsolver.maxium_central_density import maxium_central_density
from TOVsolver.solver_code import solveTOV
from TOVsolver.unit import g_cm_3, dyn_cm_2, km, Msun, e0

/Users/a9/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


We need to define the outer crust, this crust and interface is unversial for all the equation of state here, we just change the inner crust part and the core part of equation of state and that is come from the polytrope computation. Crust as below, "Tolos_crust_out.txt" is BPS crust model.

In [ ]:
#Tolos_crust_out = np.loadtxt('Tolos_crust_out.txt', delimiter='  ')
Tolos_crust_out = np.loadtxt('Tolos_crust_out.txt', delimiter=None, comments='#', usecols=(0, 1, 2, 3, 4))
eps_crust = Tolos_crust_out[:,3] * g_cm_3
pres_crust = Tolos_crust_out[:,4] * dyn_cm_2

# rho_crust_end and P_crust_end are the end of outer crust EoS and the start point of polytrope EoS
rho_crust_end = eps_crust[-1]
P_crust_end = pres_crust[-1]

# eps_core is the density array from inner crust to core, is the palce where we want to establish the polytrope EoS
# cent_densitys is the central density range of a neutron star
eps_core = np.logspace(11.7, 15.6, 1000, base=10) * g_cm_3
cent_densitys = np.logspace(14.3, 15.6, 30) * g_cm_3

### Set up prior

Next step, we need to set up the prior, first use parameters array to specify the variable name, should be consistent with what you need to call them. Second, we need to define a list to store some intermediate parameters derived from `prior_transform`. These parameters may be used in `likelihood_transform` or somewhere else and it will be stored in the final result, specifically 'd_max' in this case.
    
Define a prior transform function to define prior. Cube are set of random number from 0 to 1. This prior setting is standard set-up of UltraNest package, since we are using `UltraNest` to do nest-sampling. We provided 

`normal_Prior` and `flat_prior`

two options call from prior. Here then the Parameters prior should all set

------------------

However, since we are doing Equation of state Inference from mass radius of neutron star measurement. The center density of the star should be also sampled. Otherwise will be a partially-defined prior, did not span all parameters space, and proved to be different with full-scope inference.

This request as randomly generate a density from a EoS range, however, this process is not that trivial, since we need to determine the upper limit of the central density of neutron star ---  different equation of state will predict different upper bound, so here we need to use the prior-setting EoS parameters computing the EOS by

```sh
Polytrope.compute_EOS
```

Compute out EOS, put into

```sh
TOVsolver.maxium_central_density
```

find out Mass Radius of this equation of state, find out the last stable point of this equation of state.(first mass points that give the direvative to be negative)

In [ ]:
parameters = ['rho1', 'rho2', 'rho3', 'gamma1', 'gamma2', 'gamma3', 'gamma4', 'd1', 'd2', 'd3']
derived_param_names = ['d_max']

gamma_lo_ranges = [1, 0.1, 0.1, 0.1]
gamma_up_ranges = [1.5, 4, 8, 4]
def prior_transform(cube):
    para = cube.copy()

    # Output appropriate gamma
    # When we randomly scatter a gamma at one segement, its maxium value is constrained by the start point's pressure, density,
    # and the end point's density. Therefore in each segement, we need call 'fun_gamma_max' to get a sutiable range of gamma.
    para[0] = prior.flat_prior(0.04/0.16*e0, 0.065/0.16*e0, cube[0])
    para[1] = prior.flat_prior(1.5*e0, 8.3*e0, cube[1])
    para[2] = prior.flat_prior(para[1], 8.3*e0, cube[2])
    rho_s = [para[0], para[1], para[2], eps_core[-1]]
    gamma_s = []
    k = 0
    pt = 0
    for i in range(4):
        if i==0:
            gamma_max = Polytrope.fun_gamma_max(rho_s[i], rho_crust_end, P_crust_end)
            if gamma_up_ranges[i] < gamma_max:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_up_ranges[i], cube[3+i])
            else:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_max, cube[3+i])
            gamma_s.append(para[3+i])
            k = P_crust_end / (rho_crust_end**gamma_s[-1])
            pt = k*rho_s[0]**gamma_s[-1]
        else:
            gamma_max = Polytrope.fun_gamma_max(rho_s[i], rho_s[i-1], pt)
            if gamma_up_ranges[i] < gamma_max:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_up_ranges[i], cube[3+i])
            else:
                para[3+i] = prior.flat_prior(gamma_lo_ranges[i], gamma_max, cube[3+i])
            gamma_s.append(para[3+i])
            k = pt / (rho_s[i-1]**gamma_s[-1])
            pt = k*rho_s[i]**gamma_s[-1]

    gammas = np.array([para[3],para[4],para[5],para[6]])
    rho_ts= np.array([para[0],para[1],para[2]])
    theta = np.append(gammas, rho_ts)
    pres = Polytrope.compute_EOS(eps_core, theta)
    eps = np.hstack((eps_crust, eps_core))
    pres = np.hstack((pres_crust, pres))
    
    d_max = maxium_central_density(eps, pres, cent_densitys)
    para[7:] = 14.3 + (np.log10(d_max / g_cm_3) - 14.3) * cube[7:]

    return np.append(para, d_max) # d_max is in natural unit, but d1,d2,d3 are the result of taking the logarithm of density in g_cm3

### Set up likelihood

We need to set up a likelihood, Using standard definition way of UltraNest, that is below.

Here the likelihood is generated from a simulated mass radius measurement, which is $M = 1.97, 1.44, 2.07$ $M_{\odot}$ and $R = 10, 13, 12$ km, With a 5% Mass radius measurement uncertainty, so here 

   ```sh
   likelihood.MRlikihood_Gaussian
   ```
function will be use for our likelihood, please check [likelihood.MRlikihood_Gaussian](https://github.com/ChunHuangPhy/EoS_inference/blob/main/InferenceWorkflow/Likelihood.py) to see the original code, and more choice of likelihood.
eg:
1. If we have some real mass-radius measurements, say PSR J0030 or PSR J0740, come from NICER, a KDE kernel could be trained to feed into 

   ```sh
   likelihood.MRlikihood_kernel(eps_total,pres_total,x,d1)
   ```
set the KDE kernel as a input for this function

2. If we gain measurement from radio-timing, say only measure the neutron star mass, then

   ```sh
   likelihood.Masslikihood_Gaussian(eps_total,pres_total,x,d1)
   ```
Which will give the likelihood from single mass measurement, x is the parameters of that measurement, you should specify where this measurement mass is located and what is the sigma width of this mass measurement

In [ ]:
import pandas as pd
from scipy.interpolate import interp1d

datas = [
    [10, 1.97, 0.5, 0.0985],
    [13, 1.44, 0.65, 0.072],
    [12, 2.07, 0.6, 0.1035]
]
datas = pd.DataFrame(datas, index=[1,2,3], columns=['R', 'M', 'Rsigma', 'Msigma'])


Rvalue = datas['R'].values * km
Mvalue = datas['M'].values * Msun

R_sigma = datas['Rsigma'].values * km
M_sigma = datas['Msigma'].values * Msun

f1 = np.log(1/(R_sigma*M_sigma*(np.sqrt(2*np.pi))**2)).sum()

def from_para_to_MR_points(theta, densitys):
    pres = Polytrope.compute_EOS(eps_core, theta)
    eps = np.hstack((eps_crust, eps_core))
    pres = np.hstack((pres_crust, pres))
    unique_pressure_indices = np.unique(pres, return_index=True)[1]
    unique_pressure = pres[np.sort(unique_pressure_indices)]

    eos = interp1d(eps, pres, kind='cubic', fill_value='extrapolate')
    inveos = interp1d(unique_pressure, eps[unique_pressure_indices], kind='cubic', fill_value='extrapolate')

    MR_points = []
    for dens in densitys:
        try:
            result = solveTOV(dens, pres[20], eos, inveos)
            MR_points.append(result)
        except OverflowError as e:
            MR_points.append([0,0])
    return np.array(MR_points)

def likelihood_transform(theta):
    gammas = np.array([theta[3],theta[4],theta[5],theta[6]])
    rho_ts = np.array([theta[0],theta[1],theta[2]])
    d_s = 10 ** np.array(theta[7:-1]) * g_cm_3

    theta = np.append(gammas, rho_ts)
    Ms, Rs = from_para_to_MR_points(theta, d_s).T

    # In order to drop the unpysical parameters more efficiently, I used super Gaussian function here.
    fm = np.where(Ms>3*Msun, -(Ms-2*Msun)**20, -(Ms-Mvalue)**2/(2*M_sigma**2))
    fr = np.where(Rs>20*km, -(Rs-16*km)**20, -(Rs-Rvalue)**2/(2*R_sigma**2))

    f2 = (fm + fr).sum()
    likelihood = f1 + f2
    return likelihood

### Set up sampler

Here next, we define sampler.

Some of the sampler parameters is requested, first is step number, our choice for UltraNest sampler is slicesampler, which could easily be sliced up your total computation load, and parallelize, speed up sampling. So step as suggested by documentation of UltraNest, we use 2*len(parameters).

live_point we set 4000, it will influence the sampling precision, We suggest for 10 dimension space, maybe 5000 is a better choice, however, since my computer only have limited resources, we set 4000.

max_calls set 500000, it is how many iteration after it will stop, we suggest to set this number significantly higher, maybe 5000000, otherwise maybe will broken before the inference converging to a definite value. That result will be un-phyiscal.

derived_param_names is calculated from prior_transform and is the intermediate quantity necessary for calculating likelihood_transform. It will not be involved in the inference process.


In [ ]:
step = 2 * len(parameters)
live_point = 4000 # More parameters need more points.
max_calls = 500000 # In the example code, the result is Z=-5.7(23.50%), which means that the max_calls should be larger.

sampler = ultranest.ReactiveNestedSampler(parameters, likelihood_transform, prior_transform, log_dir='output/Polytrope_EOS', derived_param_names=derived_param_names)
sampler.stepsampler = ultranest.stepsampler.SliceSampler(
    nsteps=step,
    generate_direction=ultranest.stepsampler.generate_mixture_random_direction,
    # adaptive_nsteps=False,
    # max_nsteps=400
)

result = sampler.run(min_num_live_points=live_point,max_ncalls= max_calls)
flat_samples = sampler.results['samples']

Creating directory for new run output/run22


UnboundLocalError: local variable 'chrip_mass' referenced before assignment